In [24]:
%reload_kedro

2023-05-02 14:53:43,659 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/gpalazzo/Desktop/dev/crypto_msc_thesis
2023-05-02 14:53:43,939 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-05-02 14:53:44,250 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-05-02 14:53:44,251 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [25]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import quantstats as qs
import math

In [27]:
portfolio_initial_money = catalog.load("params:portfolio_initial_money")

2023-05-02 14:53:45,905 - kedro.io.data_catalog - INFO - Loading data from `params:portfolio_initial_money` (MemoryDataSet)...


## Buy and Hold

In [37]:
df_benchm_buyhold = catalog.load("benchmark_buyhold_strat")

2023-05-02 14:55:33,214 - kedro.io.data_catalog - INFO - Loading data from `benchmark_buyhold_strat` (ParquetDataSet)...


In [38]:
first_time = df_benchm_buyhold.date.min()
last_time = df_benchm_buyhold.date.max()

first_price = float(df_benchm_buyhold[df_benchm_buyhold["date"] == first_time]["close_px"])
last_price = float(df_benchm_buyhold[df_benchm_buyhold["date"] == last_time]["close_px"])

In [48]:
# retorno percentual
pctchg = last_price / first_price - 1

# profit nominal
qty = math.floor(portfolio_initial_money / first_price)
profit_nominal = qty * last_price - portfolio_initial_money

# sharpe
vol = np.nanstd(df_benchm_buyhold["pctchg"], ddof=1)
sharpe = pctchg / vol

In [58]:
print(f"Cumulative return: {pctchg*100}%")
print(f"Nominal profit: {profit_nominal}")
print(f"Sharpe: {sharpe}")

Cumulative return: 208.60814237114064%
Nominal profit: 311549.4
Sharpe: 506.27275375812945
